![title](pic/11.png)

![title](pic/12.png)

![title](pic/13.png)

![title](pic/14.png)

![title](pic/15.png)

In [ ]:
import numpy as np
import re
import random

In [ ]:
def loadDataSet():   #构建简单测试集
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

In [ ]:
def  createVocabList(dataset):#挑出所有出现过的字符
    vocabSet=set([ vocab for x in dataset for vocab in x ])  
    return  list(vocabSet)

In [ ]:
Dataset,label=loadDataSet()
Dataset,label

In [ ]:
a=createVocabList(Dataset)
a

In [ ]:
def setOfwords2Vec(vocabList,inputSet):        #向量化，词集模型
    returnVec=[0]*len(vocabList)
    for vocab in  inputSet:
        if vocab in vocabList:
            returnVec[vocabList.index(vocab)]=1
        else:
            print('not exist')
    return returnVec

In [ ]:
trainMat=[]
for postinDoc in Dataset:  #构建整个词集
    trainMat.append(setOfwords2Vec(a,postinDoc))

In [ ]:
trainMat

In [ ]:
def trainNB0(trainMatrix,trainCategory):      #计算每个词的条件概率
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.ones(numWords); p1Num = np.ones(numWords)      #change to ones() ，拉普拉斯平滑
    p0Denom = 2.0; p1Denom = 2.0                        #change to 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)          #change to log()
    p0Vect = np.log(p0Num/p0Denom)          #change to log()
    return p0Vect,p1Vect,pAbusive

In [ ]:
trainNB0(trainMat,label)

In [ ]:
def classifyNB(vec2Classify,p0,p1,pA):  #判别label，贝叶斯公式
    p1=sum(vec2Classify*p1)+np.log(pA)
    p0=sum(vec2Classify*p0)+np.log(1.0-pA)
    if p0>p1:return 0
    else: return 1

In [ ]:
def testingNB(testEntry=['love','my','dalmation']):  #测试功能
    listOPosts,listClasses=loadDataSet()
    myVocabList=createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfwords2Vec(myVocabList,postinDoc))
    p0,p1,pA=trainNB0(np.array(trainMat),np.array(listClasses))
    thisDoc=np.array(setOfwords2Vec(myVocabList,testEntry))
    print(testEntry,'classified as:',classifyNB(thisDoc,p0,p1,pA))

In [ ]:
testingNB()

In [ ]:
def bagOfwords2VecMN(vocabList,inputSet):
    returnVec=[0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]+=1
    return  returnVec

In [ ]:
def textParse(textstring):
    listOfTokens=re.split(r'\W',textstring)
    return [word.lower() for word in listOfTokens if len(word)>2]

In [ ]:
def spamTest():   #邮件分类
    docList=[];classList=[]  
    for i in range(1,27):
        wordList=textParse(open(r'data\email\spam\{}.txt'.format(i),encoding='gb18030',errors='ignore').read())
        docList.append(wordList)
        classList.append(1)
        wordList=textParse(open(r'data\email\ham\{}.txt'.format(i),encoding='gb18030',errors='ignore').read())
        docList.append(wordList)
        classList.append(0)
    vocabList=createVocabList(docList)
    #创建测试集
    testSet=[];testIndex=[random.randint(0,49) for i in range(10)]
    for i in testIndex:
        testSet.append(docList[i])
    trainMat=[];trainClasses=[]
    for docIndex in range(50):
        trainMat.append(setOfwords2Vec(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0,p1,pA=trainNB0(np.array(trainMat),np.array(trainClasses))
    #
    errorCount=0
    for doc in testIndex:
        wordVector=setOfwords2Vec(vocabList,docList[doc])
        if classifyNB(np.array(wordVector),p0,p1,pA)!=classList[doc]:
            errorCount+=1
    print('error:%f'%(float(errorCount)/len(testSet)))

In [ ]:
spamTest()